In [ ]:
# importar conjunto de datos
import pandas as pd
df = pd.read_csv("FANV anticoagulados 2007-21.csv")
df

### Reorganización y creación de nuevas variables

In [ ]:
# clarificar columnas
df.columns = df.columns.str.replace("á","a")
df.columns = df.columns.str.replace("é","e")
df.columns = df.columns.str.replace("í","i")
df.columns = df.columns.str.replace("ó","o")
df.columns = df.columns.str.replace("ú","u")
df.columns = df.columns.str.replace("ñ","n")
df.columns = df.columns.str.replace("(","")
df.columns = df.columns.str.replace(")","")
df.columns = df.columns.str.replace('/ mL',"")
df.columns = df.columns.str.replace("/mL","")
df.columns = df.columns.str.replace("g/dL","")
df.columns = df.columns.str.replace(" %","")
df.columns = df.columns.str.upper()
df.columns = df.columns.str.strip()
df.columns = df.columns.str.replace("º ","_")
df.columns = df.columns.str.replace("º","_")
df.columns = df.columns.str.replace(" ","_")
df.columns = df.columns.str.replace("/","_")

df.columns

In [ ]:
# quitar columnas innecesarias
df.drop(["CRITERIO_DE_GRUPO","NHC","OTRAS_DROGAS"],inplace=True,axis=1)

In [ ]:
# tipos de datos
df.dtypes

In [ ]:
# valores nulos de cada columna
df.isnull().sum()

In [ ]:
#imputar los valores nulos de la columna IMC con la media de la columna
df["IMC"].fillna(df["IMC"].mean(),inplace=True)

In [ ]:
# valores nulos en total
df.isnull().sum().sum()

In [ ]:
# cambiar el tipo de dato de las columnas Fecha inicio TAO y Fecha nacimiento a datetime
df["FECHA_INICIO_TAO"] = pd.to_datetime(df["FECHA_INICIO_TAO"],format="%d/%m/%Y")
df["FECHA_NACIMIENTO"] = pd.to_datetime(df["FECHA_NACIMIENTO"],format="%d/%m/%Y")

In [ ]:
# crear una columna Edad (en años) 
df["EDAD"] = round((df["FECHA_INICIO_TAO"] - df["FECHA_NACIMIENTO"]).dt.days/365.25,0)

In [ ]:
# crear la variable Ratio neutrofilo/linfocito
df["RATIO_NEUTROFILO/LINFOCITO"] = round(df["NEUTROFILOS"]/df["LINFOCITOS"],2)

In [ ]:
# crear la columna Anemia 
df["ANEMIA"] = 0
df.loc[(df["SEXO"] == "H") & (df["HEMOGLOBINA"] < 13),"ANEMIA"] = 1
df.loc[(df["SEXO"] == "M") & (df["HEMOGLOBINA"] < 12),"ANEMIA"] = 1

In [ ]:
# crear la columna Trombocitopenia
df["TROMBOCITOPENIA"] = 0
df.loc[df["PLAQUETAS"] < 100,"TROMBOCITOPENIA"] = 1

In [ ]:
# Edad > 60
df["EDAD_60"] = 0
df.loc[df["EDAD"] >= 60,"EDAD_60"] = 1

In [ ]:
# calcular el CHA2DS2-VASc score
df["CHA2DS2-VASC"] = 0
df.loc[df["EDAD"] >= 75,"CHA2DS2-VASC"] += 2
df.loc[(df["EDAD"] >= 65) & (df["EDAD"] < 75),"CHA2DS2-VASC"] += 1
df.loc[df["HTA"] == 1,"CHA2DS2-VASC"] += 1
df.loc[(df["ICTUS_PREVIO"] == 1) | (df["EMBOLIA_SISTEMICA_PREVIA"] == 1),"CHA2DS2-VASC"] += 2
df.loc[df["DM"] == 1,"CHA2DS2-VASC"] += 1
df.loc[(df["CARDIOPATIA_ISQUEMICA"] == 1) | (df["ARTERIOPATIA_PERIFERICA"] == 1),"CHA2DS2-VASC"] += 1
df.loc[df["SEXO"] == "M","CHA2DS2-VASC"] += 1
df.loc[df["INSUFICIENCIA_CARDIACA"] == 1,"CHA2DS2-VASC"] += 1

In [ ]:
# calcular el SAMe-TT2R2 score
df["SAME-TT2R2"] = 0
df.loc[df["SEXO"] == "M","SAME-TT2R2"] += 1
df.loc[df["EDAD"] < 60,"SAME-TT2R2"] += 1
df.loc[(df["HTA"] + df["DM"] + df["CARDIOPATIA_ISQUEMICA"] + df["ARTERIOPATIA_PERIFERICA"] + df["INSUFICIENCIA_CARDIACA"] + df["ICTUS_PREVIO"] + df["ENFERMEDAD_PULMONAR"] + (df["ALTERACION_HEPATICA"] | df["INSUFICIENCIA_RENAL"]))>2,"SAME-TT2R2"] += 1
df.loc[(df["AMIODARONA"] == 1), "SAME-TT2R2"] += 1
df.loc[df["FUMA"]==1,"SAME-TT2R2"] += 2


In [ ]:
# quitamos las columnas que no vamos a utilizar
df.drop(["FECHA_INICIO_TAO","FECHA_NACIMIENTO"],inplace=True,axis=1)

In [ ]:
# one hot encoding
dummies_neoplasia = pd.get_dummies(df['NEOPLASIA'], prefix='NEOPLASIA')
df = pd.concat([df, dummies_neoplasia], axis=1)
df.drop(["NEOPLASIA"],inplace=True,axis=1)
df.head()


In [ ]:
# label encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df["SEXO"] = le.fit_transform(df["SEXO"])

In [ ]:
# crear una nueva variable "Cualquier antiarritmico"
df["CUALQUIER_ANTIARRITMICO"] = 0
df.loc[(df["AMIODARONA"] == 1) | (df["OTROS_ANTIARRITMICOS"] == 1),"CUALQUIER_ANTIARRITMICO"] = 1

In [ ]:
# crear la variable control frecuencia
df["CONTROL_FRECUENCIA"] = 0
df.loc[(df["BETABLOQUEANTES"] == 1) | (df["SOTALOL"] == 1) | (df["VERAPAMILO"] == 1) | (df["DILTIAZEM"] == 1) | (df["DIGOXINA"] == 1),"CONTROL_FRECUENCIA"] = 1

In [ ]:
# crear la variable cualquier antiagregante
df["CUALQUIER_ANTIAGREGANTE"] = 0
df.loc[(df["ASPIRINA"] == 1) | (df["OTROS_ANTIAGREGANTES"] == 1),"CUALQUIER_ANTIAGREGANTE"] = 1

In [ ]:
# crear la variable doble antiagregacion
df["DOBLE_ANTIAGREGACION"] = 0
df.loc[(df["ASPIRINA"] == 1) & (df["OTROS_ANTIAGREGANTES"] == 1),"DOBLE_ANTIAGREGACION"] = 1

In [ ]:
# crear la variable SAMe-TT2R2 mayor o igual a 3
df['SAME-TT2R2_MAS3'] = 0
df.loc[df['SAME-TT2R2'] > 2, 'SAME-TT2R2_MAS3'] = 1
df['SAME-TT2R2_MAS3'].value_counts()

In [ ]:
# crear la variable n_comedicaciones
df["N_COMEDICACIONES"]= 0
df.loc[(df["CONTROL_FRECUENCIA"] == 1) | (df["CUALQUIER_ANTIARRITMICO"] == 1) ,"N_COMEDICACIONES"] = 1
df.loc[(df["CONTROL_FRECUENCIA"] == 1) & (df["CUALQUIER_ANTIARRITMICO"] == 1) ,"N_COMEDICACIONES"] = 2

In [ ]:
# crear la variable ictus/embolia
df["ICTUS/EMBOLIA"] = 0
df.loc[(df["ICTUS_PREVIO"] == 1) | (df["EMBOLIA_SISTEMICA_PREVIA"] == 1),"ICTUS/EMBOLIA"] = 1

In [ ]:
# crear la variable enfermedad vascular
df["ENFERMEDAD_VASCULAR"] = 0
df.loc[(df["CARDIOPATIA_ISQUEMICA"] == 1) | (df["ARTERIOPATIA_PERIFERICA"] == 1),"ENFERMEDAD_VASCULAR"] = 1

In [ ]:
# crear la variable disfuncion/dilatacion
df["DISFUNCION/DILATACION"] = 0
df.loc[(df["DISFUNCION_VENTRICULAR_EN_ECO"] == 1) | (df["TAMANO_AURICULA_IZQUIERDA"] == 1),"DISFUNCION/DILATACION"] = 1

In [ ]:
# crear la variable comorbilidades en SAME-TT2R2 mayor o igual a 2
df["SAME_MAS_DE_2"] = 0
df.loc[(df["HTA"] + df["DM"] + df["CARDIOPATIA_ISQUEMICA"]+ df["ARTERIOPATIA_PERIFERICA"]  + df["INSUFICIENCIA_CARDIACA"] + df["ICTUS_PREVIO"] + (df["ALTERACION_HEPATICA"] | df["INSUFICIENCIA_RENAL"]) + df["ENFERMEDAD_PULMONAR"] )>2,"SAME_MAS_DE_2"] = 1

In [ ]:
# crear la variable alteracion
df["ALTERACION"] = 0
df.loc[(df["ALTERACION_HEPATICA"] == 1) | (df["INSUFICIENCIA_RENAL"] == 1),"ALTERACION"] = 1

In [ ]:
# crear la variable bloqueo calcio
df["BLOQUEO_CALCIO"]=0
df.loc[(df["VERAPAMILO"]==1) | (df["DILTIAZEM"]==1),"BLOQUEO_CALCIO"]=1

In [ ]:
# copia
df_completo = df.copy()

In [ ]:
# frecuencia de las variables categóricas
import matplotlib.pyplot as plt
lista = []
for i in df_completo.columns:
    if df_completo[i].nunique() <= 10:
        lista.append(i)
for var in lista:
    print(df_completo[var].value_counts())
    print("")

### Identificación de valores anómalos por RIQ y Z-score

In [ ]:
# boxplot de todas las variables numericas con mas de 10 valores distintos
lista = []
for i in df_completo.columns:
    if df_completo[i].nunique() > 10:
        lista.append(i)
for var in lista:
    plt.figure()
    df_completo.boxplot(column=var)

In [ ]:
# mirar graficamente outliers de cada columna con IQR
lista = []
for i in df_completo.columns:
    if df_completo[i].nunique() > 10:
        lista.append(i)
for var in lista:
    q1 = df_completo[var].quantile(0.25)
    q3 = df_completo[var].quantile(0.75)
    iqr = q3 - q1
    plt.figure()
    df_completo.boxplot(column=var)
    plt.plot([0,2],[q1 - 1.5*iqr,q1 - 1.5*iqr],c="red")
    plt.plot([0,2],[q3 + 1.5*iqr,q3 + 1.5*iqr],c="red")

In [ ]:
# mirar los valores de las filas con outliers por RIQ
lista = []
outliers_arriba = []
outliers_abajo = []
for i in df_completo.columns:
    if df_completo[i].nunique() > 10:
        lista.append(i)
lista.remove("TRT_6MESES_SIN_1_MES")
lista.remove("N_DETERMINACIONES_PARA_TRT")
for var in lista:
    q1 = df_completo[var].quantile(0.25)
    q3 = df_completo[var].quantile(0.75)
    iqr = q3 - q1
    print(var)
    try:
        print((df_completo.loc[df[var] > q3 + 1.5*iqr,var]).count())
        print((df_completo.loc[df[var] < q1 - 1.5*iqr,var]).count())
        # meter en lista
        outliers_arriba.append((df_completo.loc[df[var] > q3 + 1.5*iqr,var]).count())
        outliers_abajo.append((df_completo.loc[df[var] < q1 - 1.5*iqr,var]).count())
    except:
        pass
    print("")

In [ ]:
# histograma del numero de outliers por columna con RIQ
plt.figure()
plt.bar(lista,outliers_arriba)
plt.bar(lista,outliers_abajo)
plt.xticks(rotation=90)
# titulo
plt.title("Número de outliers por variable (RIQ)")
# etiquetas de los ejes
plt.xlabel("Variables")
plt.ylabel("Numero de outliers")
# leyenda
plt.legend(["Outliers arriba","Outliers abajo"])
# hacer las xlabel mas pequeñas

# mejorar resolucion
plt.savefig("outliers1.png",bbox_inches='tight',dpi=500)

# mostrar grafico
plt.show()
    

In [ ]:
# outliers con z-score
from scipy import stats
import numpy as np
lista = []
outliers_arriba = []
outliers_abajo = []
for i in df_completo.columns:
    if df_completo[i].nunique() > 10:
        lista.append(i)
lista.remove("TRT_6MESES_SIN_1_MES")
lista.remove("N_DETERMINACIONES_PARA_TRT")
for var in lista:
    z = stats.zscore(df_completo[var])
    print(var)
    try:
        print((df_completo.loc[z > 3,var]).count())
        print((df_completo.loc[z < -3,var]).count())
        # meter en lista
        outliers_arriba.append((df_completo.loc[z > 3,var]).count())
        outliers_abajo.append((df_completo.loc[z < -3,var]).count())
    except:
        pass
    print("")

In [ ]:
# histograma del numero de outliers por columna
plt.figure()
plt.bar(lista,outliers_arriba)
plt.bar(lista,outliers_abajo)
plt.xticks(rotation=45)
# titulo
plt.title("Número de outliers por variable (Z-score)")
# etiquetas de los ejes
plt.xlabel("Variables")
plt.ylabel("Numero de outliers")
# leyenda
plt.legend(["Outliers arriba","Outliers abajo"])
# hacer las xlabel mas pequeñas

# mejorar resolucion
plt.savefig("outliers2.png",bbox_inches='tight',dpi=500)
# mostrar grafico
plt.show()

In [ ]:
# mirar que la suma de neutrofilos, linfocitos y monocitos es igual a la suma de los leucocitos (o -1)
df_completo["SUMA_LEUCOCITOS"] = df_completo["NEUTROFILOS"] + df_completo["LINFOCITOS"] + df_completo["MONOCITOS"]
df_completo["DIFERENCIA_LEUCOCITOS"] = df_completo["SUMA_LEUCOCITOS"] - df_completo["LEUCOCITOS"]

# valores absolutos de la columna "DIFERENCIA_LEUCOCITOS" que sean mayores que 1
df_completo.loc[abs(df_completo["DIFERENCIA_LEUCOCITOS"]) > 1,"DIFERENCIA_LEUCOCITOS"]

In [ ]:
# eliminar las coolumnas que no aportan informacion
df_completo.drop(["SUMA_LEUCOCITOS","DIFERENCIA_LEUCOCITOS"],inplace=True,axis=1)

In [ ]:
# a csv
df_completo.to_csv("df_completo.csv",index=False)

In [ ]:
#copia
df_sin_anomalos = df_completo.copy()

### Distribución de las variables numéricas

In [ ]:
# grafica de distribucion de todas las variables numericas
lista = []
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() > 10:
        lista.append(i)
for var in lista:
    plt.figure()
    df_sin_anomalos[var].hist()
    plt.title(var)


In [ ]:
# exportar el dataframe a un csv
df_sin_anomalos.to_csv('df_sin_anomalos.csv', index=False)

In [ ]:
# rangos
import numpy as np

q1 = df_sin_anomalos.quantile(0.25)
q3 = df_sin_anomalos.quantile(0.75)
iqr = q3 - q1

lower_limit = q1 - 1.5 * iqr
upper_limit = q3 + 1.5 * iqr

In [ ]:
lower_limit[lista], upper_limit[lista]

In [ ]:
valor_z1 = 3
valor_z2 = -3

for columna in df_sin_anomalos[lista].columns:
    media = df_sin_anomalos[columna].mean()
    desviacion_estandar = df_sin_anomalos[columna].std()
    valor_original1 = (valor_z1 * desviacion_estandar) + media
    valor_original2 = (valor_z2 * desviacion_estandar) + media
    print("Valor original para z-score de 3 en la columna", columna, ":", valor_original1)
    print("Valor original para z-score de -3 en la columna", columna, ":", valor_original2)

In [ ]:
# skewness
lista = []
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() > 10:
        lista.append(i)
for var in lista:
    print(var)
    print(df_sin_anomalos[var].skew())
    print("")

In [ ]:
# cambiar el skewness de las variables con skewness positivo y negativo
import numpy as np
lista = []
transformadas = []
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() > 10:
        lista.append(i)
for var in lista:
    if df_sin_anomalos[var].skew() > 1:
        df_sin_anomalos[var] = np.log(df_sin_anomalos[var])
        transformadas.append(var)
    elif df_sin_anomalos[var].skew() < -1:
        df_sin_anomalos[var] = np.sqrt(df_sin_anomalos[var])
        transformadas.append(var)

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))

# Iterar sobre cada variable y trazar su distribución en un subgráfico
for i, column in enumerate(transformadas):
    ax = axes[i // 3, i % 3]  # Obtener el subgráfico correspondiente
    sns.histplot(data=df_sin_anomalos[transformadas], x=column, kde=True, ax=ax)  # Trazar el gráfico de distribución
    ax.set_title(column)  # Establecer el título del subgráfico
    # quitar el xlabel
    ax.set_xlabel("")

# Ajustar el espaciado entre los subgráficos
plt.tight_layout()

# quitar los dos ultimos subplots que no se usan
fig.delaxes(axes[2,1])
fig.delaxes(axes[2,2])

# Mostrar la figura
plt.show()

In [ ]:
lista = []
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() > 10:
        lista.append(i)
for var in lista:
    print(var)
    print(df_sin_anomalos[var].skew())
    print("")

### Tratamiento de valores anómalos

In [ ]:
from scipy import stats
import numpy as np
lista = []
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() > 10:
        lista.append(i)
for var in lista:
    z = np.abs(stats.zscore(df_sin_anomalos[var]))
    print(var)
    print(df_sin_anomalos.loc[z > 3,var])
    print("")   

In [ ]:
# sacar los limites de los outliers con z-score y IQR para cada columna
lista = []
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() > 10:
        lista.append(i)
for var in lista:
    q1 = df_sin_anomalos[var].quantile(0.25)
    q3 = df_sin_anomalos[var].quantile(0.75)
    iqr = q3 - q1
    print(var)
    print(q1 - 1.5*iqr)
    print(q3 + 1.5*iqr)
    print("")

    

In [ ]:
# valores considerados anomalos por el z-score
z = np.abs(stats.zscore(df_sin_anomalos["ADE"]))
valores_anomalos = df_sin_anomalos.loc[z > 3,var]

# cambiar estos valores por el valor mas alto de los no anomalos
df_sin_anomalos.loc[z > 3,"ADE"] = df_sin_anomalos.loc[z <= 3,"ADE"].max()
print(df_sin_anomalos.loc[z <= 3,"ADE"].max())

# IMC mayor que 50 a 50
print(df_sin_anomalos.loc[df_sin_anomalos["IMC"] > 50,"IMC"].count())
df_sin_anomalos.loc[df_sin_anomalos["IMC"] > 50,"IMC"] = 50

In [ ]:
z = stats.zscore(df_sin_anomalos["ADE"])
df_sin_anomalos.loc[z <= 3,"ADE"].max()

In [ ]:
# skewness de ADE
df_sin_anomalos.loc[z > -3,"ADE"].min()

In [ ]:
df_sin_anomalos["IMC"].skew()

### Boxplot variables categóricas

In [ ]:
# boxplot con sns de las variables categoricas con TRT_6MESES_SIN_1_MES
import seaborn as sns
lista2 = []
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() <= 10:
        lista2.append(i)
for var in lista2:
    plt.figure()
    sns.boxplot(x=var,y="TRT_6MESES_SIN_1_MES",data=df_sin_anomalos)

### Pruebas de significancia variables categóricas

In [ ]:
# test de normalidad y homogeneidad para TRT_6MESES_SIN_1_MES
from scipy.stats import shapiro
from scipy.stats import levene
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() == 2 and (df_sin_anomalos[i]==0).sum() > 3 and (df_sin_anomalos[i]==1).sum() > 3:
        print(i)
        print(shapiro(df_sin_anomalos[df_sin_anomalos[i]==0]['TRT_6MESES_SIN_1_MES']))
        print(shapiro(df_sin_anomalos[df_sin_anomalos[i]==1]['TRT_6MESES_SIN_1_MES']))
        print(levene(df_sin_anomalos.loc[df_sin_anomalos[i]==0,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==1,"TRT_6MESES_SIN_1_MES"]))
        print("")
    elif df_sin_anomalos[i].nunique() == 2:
        print(i)
        print(shapiro(df_sin_anomalos[i]))
        print(levene(df_sin_anomalos.loc[df_sin_anomalos[i]==0,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==1,"TRT_6MESES_SIN_1_MES"]))
        print("")

In [ ]:
# test de mann whitney para TRT_6MESES_SIN_1_MES
from scipy.stats import mannwhitneyu
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() == 2:
        print(i)
        print(mannwhitneyu(df_sin_anomalos.loc[df_sin_anomalos[i]==0,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==1,"TRT_6MESES_SIN_1_MES"]))
        print("")

In [ ]:
# test de normalidad y homogeneidad para TRT_6MESES_SIN_1_MES
from scipy.stats import shapiro
from scipy.stats import levene
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() > 2 and df_sin_anomalos[i].nunique() <= 10:
        print(i)
        # normalidad de la variable
        print(shapiro(df_sin_anomalos[i]))
        # homogeneidad de la variable para todas las categorias
        print(levene(df_sin_anomalos.loc[df_sin_anomalos[i]==0,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==1,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==2,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==3,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==4,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==5,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==6,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==7,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==8,"TRT_6MESES_SIN_1_MES"],df_sin_anomalos.loc[df_sin_anomalos[i]==9,"TRT_6MESES_SIN_1_MES"]))
        print("")

In [ ]:
# test de kruskal wallis para TRT_6MESES_SIN_1_MES para variables categoricas con mas de dos valores unicos
from scipy.stats import kruskal
for i in df_sin_anomalos.columns:
    if df_sin_anomalos[i].nunique() > 2 and df_sin_anomalos[i].nunique() <= 10:
        print(i)
        print(kruskal(*[group["TRT_6MESES_SIN_1_MES"].values for name, group in df_sin_anomalos.groupby(i)]))
        print("")

### Estandarización

In [ ]:
# estandardizar las variables numericas 
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

for col in df_sin_anomalos.columns:
    if len(df_sin_anomalos[col].unique()) > 10:
        if col != 'TRT_6MESES_SIN_1_MES':
            df_sin_anomalos[col] = scaler.fit_transform(df_sin_anomalos[col].values.reshape(-1,1))


In [ ]:
# cargar df_val 
df_val = pd.read_csv("df_val.csv")

# aplicar el scaler a df_val
for col in df_val.columns:
    if len(df_val[col].unique()) > 10:
        if col != 'TRT':
            df_val[col] = scaler.transform(df_val[col].values.reshape(-1,1))

# exportar df_val a csv
df_val.to_csv("df_val_final.csv",index=False)

In [ ]:
# quitar las columnas que no se usan
df_sin_anomalos.drop(["N_DETERMINACIONES_PARA_TRT","CHA2DS2-VASC"], axis=1, inplace=True)

In [ ]:
# renombrar la columna TRT_6MESES_SIN_1_MES a TRT
df_sin_anomalos.rename(columns={"TRT_6MESES_SIN_1_MES":"TRT"},inplace=True)

In [ ]:
variables_numericas = ["TRT","IMC","N_PASTILLAS_D_INICIO","ADE","LEUCOCITOS","NEUTROFILOS","LINFOCITOS","MONOCITOS","HEMOGLOBINA","PLAQUETAS","EDAD","RATIO_NEUTROFILO/LINFOCITO"]

In [ ]:
df_sin_anomalos[variables_numericas]

### Correlación de Pearson y Spearman

In [ ]:
# correlacion de pearson y spearman heatmap
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
a = sns.heatmap(df_sin_anomalos[variables_numericas].corr(method='spearman'), annot=True)
plt.title('Correlación de Spearman')
# variable TRT de color rojo
a.get_yticklabels()[0].set_color("red")
a.get_xticklabels()[0].set_color("red")
# linea separadora de la variable TRT mas gorda
a.axhline(y=1, color='w', linestyle='-')
# mas gorda
a.axvline(x=1, color='w', linestyle='-')
# cambiar los label de los ejes
plt.show()

# correlacion de pearson y spearman heatmap
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
b = sns.heatmap(df_sin_anomalos[variables_numericas].corr(method='pearson'), annot=True)
plt.title('Correlación de Pearson')
b.get_yticklabels()[0].set_color("red")
b.get_xticklabels()[0].set_color("red")
# linea separadora de la variable TRT mas gorda
b.axhline(y=1, color='w', linestyle='-')
# mas gorda
b.axvline(x=1, color='w', linestyle='-')
plt.show()


In [ ]:
# sacar el p-value de la correlacion de pearson para cada variable
# importar la libreria
from scipy.stats import pearsonr, spearmanr
print("PEARSON")
for col in variables_numericas:
    print(col, pearsonr(df_sin_anomalos[col], df_sin_anomalos['TRT'])[1])
    print()
print()
# sacar el p-value de la correlacion de spearman para cada variable
print("SPEARMAN")
for col in variables_numericas:
    print(col, spearmanr(df_sin_anomalos[col], df_sin_anomalos['TRT'])[1])
    print()

In [ ]:
# graficar los p-valores de la correlacion de spearman y pearson
# importar la libreria
import matplotlib.pyplot as plt
# crear una lista con los p-valores de la correlacion de spearman
spearman_pval = []
for col in variables_numericas:
    spearman_pval.append(spearmanr(df_sin_anomalos[col], df_sin_anomalos['TRT'])[1])
# crear una lista con los p-valores de la correlacion de pearson
pearson_pval = []
for col in variables_numericas:
    pearson_pval.append(pearsonr(df_sin_anomalos[col], df_sin_anomalos['TRT'])[1])
# crear una lista con los nombres de las variables
variables = []
for col in variables_numericas:
    variables.append(col)

# crear un dataframe con los p-valores de la correlacion de spearman y pearson
df_pval = pd.DataFrame({"Variable":variables,"Spearman":spearman_pval,"Pearson":pearson_pval})

In [ ]:
df_pval

In [ ]:
variables.remove("TRT")
variables.remove("LEUCOCITOS")
variables.remove("RATIO_NEUTROFILO/LINFOCITO")

In [ ]:
# Configuración de la figura
fig, ax = plt.subplots()
index = range(len(variables))
bar_width = 0.35

# Gráfico de barras para Spearman
rects1 = ax.bar(index, spearman_pval, bar_width, color='b', label='Spearman')

# Gráfico de barras para Pearson
rects2 = ax.bar([i + bar_width for i in index], pearson_pval, bar_width,color = "C1", label='Pearson')

# Configuración de ejes y etiquetas
ax.set_xlabel('Variables')
ax.set_ylabel('Valor p')
ax.set_title('Comparación de valores p: Spearman vs Pearson')
ax.set_xticks([i + bar_width/2 for i in index])
ax.set_xticklabels(variables)
ax.legend()

# x labels en vertical
plt.xticks(rotation=90)
# mas pequeñas
plt.tick_params(axis='x', labelsize=8)

# linea horizontal en 0.15
plt.axhline(y=0.2, color='r', linestyle='--')

# Mostrar la figura
plt.tight_layout()
plt.show()

In [ ]:
# renombrar NEOPLASIA_0 a NEOPLASIA_NO
df_sin_anomalos.rename(columns={"NEOPLASIA_0":"NEOPLASIA_NO"},inplace=True)
df_sin_anomalos.rename(columns={"NEOPLASIA_1":"NEOPLASIA_ANTECEDENTES"},inplace=True)

In [ ]:
df_pca = df_sin_anomalos.copy()

In [ ]:
df_total = df_sin_anomalos.copy()

In [ ]:
df_total.drop(["VERAPAMILO","ALTERACION_HEPATICA","SANGRADO_PREVIO","DILTIAZEM","DEMENCIA","TROMBOCITOPENIA","DIGOXINA","ALCOHOL",
                      "NEOPLASIA_2","SOTALOL","EMBOLIA_SISTEMICA_PREVIA","DOBLE_ANTIAGREGACION",
                      "RATIO_NEUTROFILO/LINFOCITO","LEUCOCITOS","NEOPLASIA_ANTECEDENTES","CONTROL_FRECUENCIA",
                      "CUALQUIER_ANTIAGREGANTE","BLOQUEO_CALCIO","CUALQUIER_ANTIARRITMICO","ALTERACION","SAME-TT2R2",
                      "FUMA","CARDIOPATIA_ISQUEMICA","TAMANO_AURICULA_IZQUIERDA","ICTUS_PREVIO","N_COMEDICACIONES","EDAD_60"], axis=1, inplace=True)

In [ ]:
df_final = df_total.copy()

In [ ]:
# exportar el dataframe final
df_final.to_csv('df_final.csv', index=False)

### PCA

In [ ]:
variables_numericas = []
for i in df_pca.columns:
    if len(df_pca[i].unique()) > 10:
        if i != 'TRT':
            variables_numericas.append(i)

In [ ]:


# Supongamos que tienes una matriz de características llamada 'features' y un arreglo de la variable objetivo 'TRT'

# Crear una instancia de PCA
pca = PCA(n_components=2)  # Definir el número de componentes principales deseados

features=df_pca[variables_numericas]

# Realizar el PCA en las características
pca_features = pca.fit_transform(features)

# Crear un DataFrame con los datos de PCA y TRT
data = pd.DataFrame(data=pca_features, columns=['Componente 1', 'Componente 2'])

data['TRT'] = 0
data.loc[df_final['TRT']>60,'TRT'] = 1

# Configurar el estilo del gráfico
sns.set(style="ticks")

# Crear el scatter plot
sns.scatterplot(data=data, x='Componente 1', y='Componente 2', hue='TRT', palette='Set1')

# Agregar etiquetas y título
plt.xlabel('Componente 1')
plt.ylabel('Componente 2')
plt.title('Scatter plot del PCA separado por TRT')

# Mostrar el gráfico
plt.show()

In [ ]:
# varianza explicada
pca.explained_variance_ratio_

In [ ]:
variables_numericas 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

pca = PCA(n_components=9)  # Definir el número de componentes principales deseados

features=df_pca[variables_numericas]

# Realizar el PCA en las características
pca_features = pca.fit_transform(features)

In [ ]:
# bar plot de la varianza explicada (acumulado)
plt.figure()
plt.bar(range(1,10),pca.explained_variance_ratio_.cumsum())
plt.xlabel("Componentes")
plt.ylabel("Varianza explicada")
plt.title("Varianza explicada acumulada")
plt.show()

In [ ]:
PC1 = pca_features[:,0]
PC2 = pca_features[:,1]
ldngs = pca.components_

In [ ]:
scalePC1 = 1.0/(PC1.max() - PC1.min())
scalePC2 = 1.0/(PC2.max() - PC2.min())

In [ ]:
data = pd.DataFrame(data=pca_features, columns=['Componente 1', 'Componente 2','Componente 3','Componente 4','Componente 5','Componente 6','Componente 7','Componente 8','Componente 9'])

In [ ]:
data['TRT'] = 0
data.loc[df_final['TRT']>=60,'TRT'] = 1

In [ ]:
fig, ax = plt.subplots(figsize=(14, 9))
 
for i, feature in enumerate(features):
    ax.arrow(0, 0, ldngs[0, i], 
             ldngs[1, i], 
             head_width=0.03, 
             head_length=0.03,
             color="purple")
    ax.text(ldngs[0, i] * 1.15, 
            ldngs[1, i] * 1.15, 
            feature, fontsize = 10,
            color="black")
 
scatter = ax.scatter(PC1 * scalePC1, 
                     PC2 * scalePC2, 
                     c=data['TRT'], 
                     cmap='viridis')

# cambiar color de los puntos a azul claro y oscuro
scatter.set_facecolor('none')

# set edge color a azul claro y oscuro
scatter.set_edgecolor(['lightblue' if x == 0 else 'orange' for x in data['TRT']])
 
ax.set_xlabel('PC1', fontsize=20)
ax.set_ylabel('PC2', fontsize=20)

# leyenda de los puntos (0 lightblue, 1 orange)
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', markeredgecolor=color, markersize=10) for color in ['lightblue', 'orange']]
labels = ['< 60%', '>= 60%']
legend1 = ax.legend(legend_elements, labels, loc="upper right", title="TRT")

ax.add_artist(legend1)